In [95]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from joblib import dump

In [81]:
df = pd.read_csv(
    "GME.csv", index_col="Date", parse_dates=True, infer_datetime_format=True
)

In [82]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-07,17.000000,17.500000,16.219999,16.350000,16.350000,7386300
2020-12-08,16.370001,17.209999,15.930000,16.940001,16.940001,16120200
2020-12-09,13.920000,14.730000,13.230000,13.660000,13.660000,24357900
2020-12-10,13.120000,14.410000,13.050000,14.120000,14.120000,7558900
2020-12-11,13.910000,14.000000,13.020000,13.310000,13.310000,7496900


In [84]:
y = np.where(df["Close"].shift(-1) > df["Close"], 1, -1)

In [85]:
df["Open-Close"] = df.Open - df.Close
df["High-Low"] = df.High - df.Low
X = df[["Open-Close", "High-Low", "Volume"]]

In [86]:
split_percentage = 0.8
split = int(split_percentage * len(df))

In [87]:
X_train = X[:split]
y_train = y[:split]

In [88]:
X_test = X[split:]
y_test = y[split:]

In [89]:
cls = SVC().fit(X_train, y_train)

In [90]:
accuracy_train = accuracy_score(y_train, cls.predict(X_train))
accuracy_test = accuracy_score(y_test, cls.predict(X_test))

In [91]:
print("\nTrain Accuracy:{: .2f}%".format(accuracy_train * 100))
print("Test Accuracy:{: .2f}%".format(accuracy_test * 100))


Train Accuracy: 57.50%
Test Accuracy: 50.98%


In [92]:
df["signal"] = cls.predict(X)

In [93]:
cls.predict(np.array([5, 0, 1000]).reshape(1, -1))

array([-1])

In [94]:
df["signal"].value_counts()

-1    224
 1     27
Name: signal, dtype: int64

In [96]:
dump(cls, "model.joblib")

['model.joblib']

In [97]:
df = pd.read_csv(
    "AAPL.csv", index_col="Date", parse_dates=True, infer_datetime_format=True
)

In [98]:
def featurize(df: pd.DataFrame) -> pd.DataFrame:
    df["Open-Close"] = df.Open - df.Close
    df["High-Low"] = df.High - df.Low
    return df[["Open-Close", "High-Low", "Volume"]]

In [100]:
df["signal"] = cls.predict(featurize(df))

In [105]:
df["signal"].value_counts()

 1    141
-1    110
Name: signal, dtype: int64